In [ ]:
"""
Model
=====
We are using the yolov7 model, and leveraging the implementation from
the "official" yolov7 repo here: https://github.com/WongKinYiu/yolov7. The repo
has the code to do detection, training, generating various metrics with graphs,
and the pre-trained weights.

Dataset
=======
We are using the BDD100k dataset. The train, val, test sets have 70K,10K, 20K
images respc. The train and val sets come with a json file each, with the
annotations.

Data preparation
================
The json file has been converted offline into the yolov7 format.
yolov7 format is as follows:
1 file per image.
Within that file, 1 line for each detected object. Each detected object will
be represented by its numberic class_ID, and its bounding box as follows:
  class_ID, x_centroid, y_centroid, width, height
The centroids, width, and height are normalized using the image dimensions.

Training
========
We use the yolov7.pt weights, and the model has been configured to detect the
10 classes called out here: https://doc.bdd100k.com/format.html#object-detection
A new file yolov7/data/custom.yaml is used to point to the data used for
training.
A new file training/custom_yolov7.yaml is used to configure the
model to recogmnize the 10 classes

Detection/inference
===================
After training we use the generated best.pt weights for detection.
Note the last 5 cells where we do a detection.
The logs emitted just below those cells list how many of each class were
detected. It also points to the path where the image_with_detection is placed.
Those are already opened in this notebook, and you can see the detections.
"""


'\nModel\n=====\nWe are using the yolov7 model, and leveraging the implementation from\nthe "official" yolov7 repo here: https://github.com/WongKinYiu/yolov7. The repo\nhas the code to do detection, training, generating various metrics with graphs,\nand the pre-trained weights.\n\nDataset\n=======\nWe are using the BDD100k dataset. The train, val, test sets have 70K,10K, 20K\nimages respc. The train and val sets come with a json file each, with the\nannotations.\n\nData preparation\n================\nThe json file has been converted offline into the yolov7 format.\nyolov7 format is as follows:\n1 file per image.\nWithin that file, 1 line for each detected object. Each detected object will\nbe represented by its numberic class_ID, and its bounding box as follows:\n  class_ID, x_centroid, y_centroid, width, height\nThe centroids, width, and height are normalized using the image dimensions.\n\nTraining\n========\nWe use the yolov7.pt weights, and the model has been configured to detect th

In [ ]:
!nvidia-smi

Mon Aug  7 04:40:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.23 MiB | 11.49 MiB/s, done.
Resolving deltas: 100% (511/511), done.


In [ ]:
!pip install -r /content/yolov7/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.7 MB/s eta 0:00:00


In [ ]:
%cd yolov7

/content/yolov7


In [ ]:
!python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg

Namespace(weights=['yolov7.pt'], source='inference/images/horses.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.5625MB)

100% 72.1M/72.1M [00:05<00:00, 14.0MB/s]

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6652669 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[a

In [ ]:
"""
Setup and cleanup code to get the data zip files from Google Drive,
and extract them for use from this Google Colab Notebook.
"""
def extractzip(zip_path, extract_path):
  import os
  import zipfile

  # print('zipfile: ', zip_path)
  # print('extract path: ', extract_path)

  os.makedirs(extract_path, exist_ok = True)

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

def setup_data():
  # MOUNT_PATH's value is where the drive gets mounted by convention.
  # From there onwards, the file paths point to where the zip files are on
  # my Google Drive.
  # The zip file will be extracted to EXTRACT_PATH. We can then see in
  # colab using the Files view on the left.
  MOUNT_PATH       = "/content/drive"
  BDD_ZIPFILE_PATH = MOUNT_PATH + "/MyDrive/AIML_HYD_20/Capstone_Project/BDD100K/BDD100K_yolov7.zip"
  EXTRACT_PATH     = "/content"

  from google.colab import drive
  drive.mount(MOUNT_PATH)
  extractzip(BDD_ZIPFILE_PATH, EXTRACT_PATH)
  drive.flush_and_unmount()


In [ ]:
setup_data()


Mounted at /content/drive


In [ ]:
train_img_path = "/content/images/train"
val_img_path   = "/content/images/val"
test_img_path  = "/content/images/test"


In [ ]:
%cd /content

/content


In [ ]:
import os

In [ ]:
#Training images
with open("/content/train.txt", "w") as f:
  img_list = os.listdir(train_img_path)
  for img in img_list:
    f.write(os.path.join(train_img_path,img+'\n'))

In [ ]:
# Validation Image
with open("/content/val.txt", "w") as f:
  img_list = os.listdir(val_img_path)
  for img in img_list:
    f.write(os.path.join(val_img_path,img+'\n'))

In [ ]:
# Validation Image
with open("/content/test.txt", "w") as f:
  img_list = os.listdir(test_img_path)
  for img in img_list:
    f.write(os.path.join(test_img_path,img+'\n'))

In [ ]:
"""
The below step is not required for the following reason:
In an earlier step when we ran a detect on the horses images,
the yolov7.pt model was automatically downloaded.
If were to not do that step, then we must explicitly download the
model as shown below.
"""

# # download the yolov7 weights
# %cd /content/yolov7
# !wget "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt"

'\nThe below step is not required for the following reason:\nIn an earlier step when we ran a detect on the horses images,\nthe yolov7.pt model was automatically downloaded.\nIf were to not do that step, then we must explicitly download the\nmodel as shown below.\n'

In [ ]:
%cp /content/yolov7/data/coco.yaml /content/yolov7/data/custom.yaml

# use content as below
"""
train: /content/train.txt
val: /content/val.txt

# number of classes
nc: 10

# class names
names: [ 'pedestrian',
         'rider',
         'car',
         'truck',
         'bus',
         'train',
         'motorcycle',
         'bicycle',
         'traffic light',
         'traffic sign' ]
"""


"\ntrain: /content/train.txt\nval: /content/val.txt\n\n# number of classes\nnc: 10\n\n# class names\nnames: [ 'pedestrian',\n         'rider',\n         'car',\n         'truck',\n         'bus',\n         'train',\n         'motorcycle',\n         'bicycle',\n         'traffic light',\n         'traffic sign' ]\n"

In [ ]:
%cp /content/yolov7/cfg/training/yolov7.yaml /content/yolov7/cfg/training/custom_yolov7.yaml

# Change nc as below
"""
nc: 10  # number of classes
"""


'\nnc: 10  # number of classes\n'

In [ ]:
%cd yolov7

/content/yolov7


In [ ]:
def copy_files_to_drive():

    from google.colab import drive
    import shutil

    # Mount Google Drive
    drive.mount('/content/drive')

    # Source path of the best.pt file
    src_weights_file = '/content/yolov7/runs/train/exp3/weights/best.pt'
    src_confmatrix_file = '/content/yolov7/runs/train/exp3/confusion_matrix.png'
    src_f1curve_file = '/content/yolov7/runs/train/exp3/F1_curve.png'
    src_pcurve_file = '/content/yolov7/runs/train/exp3/P_curve.png'
    src_prcurve_file = '/content/yolov7/runs/train/exp3/PR_curve.png'
    src_rcurve_file = '/content/yolov7/runs/train/exp3/R_curve.png'
    src_resultspng_file = '/content/yolov7/runs/train/exp3/results.png'
    src_resultstxt_file = '/content/yolov7/runs/train/exp3/results.txt'

    # Destination directory on Google Drive
    destination_dir = '/content/drive/MyDrive/AIML_HYD_20/Capstone_Project/FinalWeights/'

    # Copy the file to the destination directory
    shutil.copy(src_weights_file, destination_dir)

    # Unmount Google Drive
    drive.flush_and_unmount()

In [ ]:
!python train.py --batch 48 --cfg cfg/training/custom_yolov7.yaml --epochs 45 --data /content/yolov7/data/custom.yaml --weights 'yolov7.pt' --device 0
copy_files_to_drive()

2023-08-07 05:09:55.340763: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-07 05:09:55.398683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 05:09:56.323790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.5625MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/custom_yolov7.yaml', data='/content/yolov7/data/custom.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=45, batch_

KeyboardInterrupt: ignored

In [ ]:
!pwd


/content/yolov7


In [ ]:
# !python detect.py --weights /content/yolov7/runs/train/exp/weights/best.pt --source /content/images/test/cabc30fc-e7726578.jpg
# !python detect.py --weights /content/yolov7/runs/train/exp/weights/best.pt --source /content/images/test/cabc30fc-eb673c5a.jpg
# !python detect.py --weights /content/yolov7/runs/train/exp/weights/best.pt --source /content/images/test/cabc30fc-fd79926f.jpg
# !python detect.py --weights /content/yolov7/runs/train/exp/weights/best.pt --source /content/images/test/cabc9045-1b8282ba.jpg
# !python detect.py --weights /content/yolov7/runs/train/exp/weights/best.pt --source /content/images/test/cabc9045-581f64de.jpg


In [ ]:
from google.colab import drive
import shutil
#drive.mount('/content/drive')
#drive.flush_and_unmount()

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Source path of the best.pt file
src_weights_file = '/content/yolov7/runs/train/exp3/weights/best.pt'
src_confmatrix_file = '/content/yolov7/runs/train/exp3/confusion_matrix.png'
src_f1curve_file = '/content/yolov7/runs/train/exp3/F1_curve.png'
src_pcurve_file = '/content/yolov7/runs/train/exp3/P_curve.png'
src_prcurve_file = '/content/yolov7/runs/train/exp3/PR_curve.png'
src_rcurve_file = '/content/yolov7/runs/train/exp3/R_curve.png'
src_resultspng_file = '/content/yolov7/runs/train/exp3/results.png'
src_resultstxt_file = '/content/yolov7/runs/train/exp3/results.txt'

# Destination directory on Google Drive
destination_dir = '/content/drive/MyDrive/AIML_HYD_20/Capstone_Project/FinalWeights/'

# Copy the file to the destination directory
shutil.copy(src_weights_file, destination_dir)

# Unmount Google Drive
drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
copy model
"""
from google.colab import drive
drive.mount("/content/drive")
%cp /content/drive/MyDrive/AIML_HYD_20/Capstone_Project/FinalWeights/best.pt /content/yolov7
%cp /content/drive/MyDrive/AIML_HYD_20/Capstone_Project/traffic.mp4 /content/.
drive.flush_and_unmount()


Mounted at /content/drive


In [ ]:
!python detect.py --weights best.pt --source /content/images/test/cabc30fc-e7726578.jpg
!python detect.py --weights best.pt --source /content/images/test/cabc30fc-eb673c5a.jpg
!python detect.py --weights best.pt --source /content/images/test/cabc30fc-fd79926f.jpg
!python detect.py --weights best.pt --source /content/images/test/cabc9045-1b8282ba.jpg
!python detect.py --weights best.pt --source /content/images/test/cabc9045-581f64de.jpg

Namespace(weights=['best.pt'], source='/content/images/test/cabc30fc-e7726578.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.5625MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36530318 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is

In [ ]:
!python detect.py --weights best.pt --source /content/traffic.mp4

Namespace(weights=['best.pt'], source='/content/traffic.mp4', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.5625MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36530318 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (